# CMA for CartPole

In [1]:
import gym
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cma
import numpy as np
import math
import rpyc
from IPython import display
import time
from sklearn.linear_model import LinearRegression

C:\Users\Staris\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
try:
    ROBOT_HOSTNAME = "192.168.137.132"
    conn = rpyc.classic.connect(ROBOT_HOSTNAME)
    conn.execute("print('Hello Slave. I am your master!')")
except:
    raise Exception('No conection to rpyc server on robot possible! Is the robot conneced? Is the rpyc server started?')
    


In [3]:
class RobotControler:
    def __init__(self, connection):
        motor_module = connection.modules['ev3dev2.motor']
        self.motors = [motor_module.LargeMotor('outB'), motor_module.LargeMotor('outC')]
        sensors = conn.modules['ev3dev2.sensor.lego'] 
        self.gyro = sensors.GyroSensor('in2')
        
        self.up_position_degrees = -90 # position of legs fully in front
        self.maxspeed = 80 # maximum speed of leg movement
        self.minspeed = 3 # minimum speed of leg movement
        self.angle_normalizer = 0.01
        self.rate_normalizer = 0.01
        
        
    
    def reset(self):
        """ resets the robot, performs calibration and maintainance, return when robot is reset.
        Called at the start of each episode"""
        
        # move legs to base position
        self._move_motors_to_pos(0, 20)
        self._wait_until_no_standstill()
        _, angle = self.get_rate_and_angle()
        if angle >= 0.01:
            print('calibrating gyro')
            self._calibrate_gyro()
        return
    
    def get_rate_and_angle(self):
        """ returns the rate and angle of the robot movement"""
        angle, rate = self.gyro.rate_and_angle
        return (rate * self.rate_normalizer, angle * self.angle_normalizer)
    
    def get_current_leg_pos(self):
        """ returns the curent leg pos, on a [-1, 1] interval"""
        return self.motors[0].position /  self.up_position_degrees
    
    def move_legs(self, pos):
        """ instructs the robot where to move its legs to, on a [-1, 1] interval. Function returns immediatly."""
        pos = max(-1., min(pos, 1.))
        pos_now = self.get_current_leg_pos()
        
        s = abs(pos - pos_now) / 2 *  self.maxspeed
        s = max(self.minspeed, min(s,  self.maxspeed))
        
        self._move_motors_to_pos(pos, s)
        return
    
    def _move_motors_to_pos(self, pos, speed):
        """
        moves the motors to a position [-1, 1] at speed [0,100].
        """
        assert -1. <= pos <= 1.,\
                    "{} is an invalid position, must be between -1 and 1 (inclusive)".format(pos)
        assert 0. <= speed <= 100.,\
                    "{} is an invalid position, must be between -1 and 1 (inclusive)".format(pos)
        for m in self.motors:
                m.on_to_position(speed, position=pos*self.up_position_degrees, brake=True, block=False) # moves motors, non blocking.
                
    def _calibrate_gyro(self):
        self.gyro.mode='GYRO-CAL'
        time.sleep(1)
        self.gyro.mode="GYRO-ANG"
        
    def _wait_until_no_standstill(self):
        """
        waits until the gyro acceleration does not change for 3 seconds
        """
        change = 9999
        while abs(change) > 0.01: 

            rates = []
            times = []
            # sample for one second
            t0 = t = time.time()
            while t - t0 < 3:
                _, rate = self.gyro.rate_and_angle
                rates.append(rate)
                t = time.time()
                times.append(t)


            # check change in accel with linreg
            rates = abs(np.array(rates)) # take absolute rates so swinging doesnt cancel each other out
            times = np.array(times)
            lr = LinearRegression()
            lr.fit(times[:, np.newaxis], rates)  
            change = lr.coef_
            print("Waiting to stand still. Avg acceleration in previous 3s: {}".format(change[0]))

        return

In [7]:
# Helper functions for setting the parameters of the neural network
def number_of_trainable_parameters(session):
    variables_names = [v.name for v in tf.trainable_variables()]
    values = session.run(variables_names)    
    number_of_parameters = 0
    for v in values:
        number_of_parameters += v.size
    return number_of_parameters

def set_trainable_parameters(session, parameter_vector):
    assert number_of_trainable_parameters(session) == parameter_vector.size, \
      'number of parameters do not match: %r vs. %r' % (number_of_trainable_parameters(session), parameter_vector.size)
    variables_names = [v.name for v in tf.trainable_variables()]
    values = session.run(variables_names) 
    idx = 0
    for k, v in zip(tf.trainable_variables(), values):    
        new_value = parameter_vector[idx:idx + v.size].reshape(v.shape)
        k.load(new_value, session)  # load does not add a new node to the graph
        idx += v.size

In [67]:
# Simple evaluation function, just considering a single (random) start state
def swing_motion(x, sess):
    set_trainable_parameters(sess, x)
    
    rc.reset()
    
    R = 0
    
    starttime = time.time()
    while time.time()-starttime <30 :
        state = np.array((*rc.get_rate_and_angle(),rc.get_current_leg_pos()))
        
        out = sess.run(action, {inputs: state.reshape(1,*state.shape)})[0][0]
        rc.move_legs(out)
        
        reward = abs(rc.get_rate_and_angle()[0])
        R = max([R,reward])
            
    print("total reward for session: ",R)
    return -R

In [55]:

state_space_dimension = 3
number_of_actions = 1
number_of_hidden_neurons = 2

rc = RobotControler(conn)


In [69]:
tf.reset_default_graph()

# Define policy network mapping state to action
with tf.name_scope('policy_network'):
    inputs = tf.placeholder(shape=[1, state_space_dimension],dtype=tf.float32) 
    hidden = tf.layers.dense(inputs, number_of_hidden_neurons, activation=tf.tanh, use_bias=True)
    action = tf.layers.dense(hidden, number_of_actions, activation=tf.tanh, use_bias=False)

# Do the learning
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    sess.graph.finalize()  # graph is read-only after this statement
    initial_weights = np.random.normal(0, 0.1, number_of_trainable_parameters(sess))
    res = cma.fmin(swing_motion, initial_weights, 1, {'maxfevals': 500, 'ftarget':-199.9,}, args=([sess]))

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=614298, Mon Jan 14 16:54:01 2019)
Waiting to stand still. Avg acceleration in previous 3s: -1.6021253973568321
Waiting to stand still. Avg acceleration in previous 3s: -0.17722157432097704
Waiting to stand still. Avg acceleration in previous 3s: 0.022444038837432827
Waiting to stand still. Avg acceleration in previous 3s: -0.07324313104956463
Waiting to stand still. Avg acceleration in previous 3s: -0.07364752391957616
Waiting to stand still. Avg acceleration in previous 3s: 0.040762226325504844
Waiting to stand still. Avg acceleration in previous 3s: 0.00115711295319298
calibrating gyro
total reward for session:  0.02
Waiting to stand still. Avg acceleration in previous 3s: -1.4506502816941156
Waiting to stand still. Avg acceleration in previous 3s: 0.0
total reward for session:  0.07
Waiting to stand still. Avg acceleration in previous 3s: -2.331021294057123
Waiting to stand still. Avg acceleration in previous 3s: -0.201932262

Waiting to stand still. Avg acceleration in previous 3s: -0.2596535910740928
Waiting to stand still. Avg acceleration in previous 3s: 0.17631631910169993
Waiting to stand still. Avg acceleration in previous 3s: -0.1311828592841517
Waiting to stand still. Avg acceleration in previous 3s: -0.0060881029492638435
total reward for session:  0.09
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.700000000000000e-01 1.0e+00 9.15e-01  9e-01  9e-01 10:29.7
Waiting to stand still. Avg acceleration in previous 3s: -1.736481362033096
Waiting to stand still. Avg acceleration in previous 3s: 0.061168790158396755
Waiting to stand still. Avg acceleration in previous 3s: -0.18653586204229616
Waiting to stand still. Avg acceleration in previous 3s: 0.05454738587563601
Waiting to stand still. Avg acceleration in previous 3s: 0.08976689480934821
Waiting to stand still. Avg acceleration in previous 3s: -0.029781747913931175
Waiting to stand still. Avg acceleration in p

KeyboardInterrupt: 